In [3]:
import sys
sys.path.append('../')

In [4]:
import os
import time
import torch
from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig, GSTConfig
from TTS.tts.datasets import load_tts_samples
from TTS.utils.audio import AudioProcessor
from TTS.trainer_windows import Trainer, TrainingArgs

# Old Tacotron Imports
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.configs.shared_configs import GSTConfig

# Style Tacotron Imports
from TTS.tts.models.styletacotron2 import StyleTacotron2
from TTS.tts.configs.style_tacotron2_config import StyleTacotronConfig
from TTS.style_encoder.configs.style_encoder_config import StyleEncoderConfig

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\torchaudio\backend\utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 to '


In [5]:
seed_everything(42)
output_path = './'

# init configs
dataset_config = BaseDatasetConfig(
    name="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "D:/Mestrado/Emotion Audio Synthesis (TTS)/repo_final/pt_etts/data/LJSpeech\LJSpeech-1.1")
)

audio_config = BaseAudioConfig(
    sample_rate=22050,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

style_config = StyleEncoderConfig(se_type="gst")

config = StyleTacotronConfig(  # This is the config that is saved for the future use
    style_encoder_config = style_config,
    audio=audio_config,
    batch_size=64,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    r=6,
    gradual_training=[[0, 6, 64], [10000, 4, 32], [50000, 3, 32], [100000, 2, 32]],
    double_decoder_consistency=True,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=True,
    mixed_precision=False,
    output_path=output_path,
    datasets=[dataset_config],
)
# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True)

# init model
model = StyleTacotron2(config)

# init the trainer and 🚀
trainer = Trainer(
    TrainingArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)
# Data loader
trainer.train_loader = trainer.get_train_dataloader(trainer.training_assets,trainer.train_samples, verbose=True)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 | > Found 13100 files in D:\Mestrado\Emotion Audio Synthesis (TTS)\repo_final\pt_etts\data\LJSpeech\LJSpeech-1.1
 > Using CUDA:  True
 > Number of GPUs:  1

 > Model has 52676308 parameters

 > DataLoader initialization
 | > Use phonemes: True
   | > phoneme language: en-us
 | > Numb

In [31]:
# Generating fake tensors

## phoneme tensor (b,L) , b = 4, L = 5
inputs_ = torch.randint(10, (4,5)).type(torch.long)
inputs_len = torch.ones(4).type(torch.long)*5

inputs_, inputs_len, inputs_.shape, inputs_len.shape

(tensor([[3, 2, 0, 7, 2],
         [6, 9, 1, 1, 0],
         [4, 2, 5, 4, 1],
         [8, 8, 0, 6, 5]]),
 tensor([5, 5, 5, 5]),
 torch.Size([4, 5]),
 torch.Size([4]))

In [32]:
inputs_ = trainer.model.embedding(inputs_.cuda())
inputs_.shape

torch.Size([4, 5, 512])

In [35]:
enc_out = trainer.model.encoder(inputs_.transpose(1, 2), inputs_len.cuda())
enc_out.shape # (b, l, dim)

torch.Size([4, 5, 512])

In [44]:
mels_spec = torch.rand((4, 50, 80))
speaker_embedding= None
input_args = [mels_spec, speaker_embedding]
gst_outputs = trainer.model.style_encoder_layer(input_args)

In [45]:
gst_outputs.shape

torch.Size([4, 50, 336])

In [46]:
trainer.model.gst_compute_style_embedding(enc_out, mels_spec)

RuntimeError: The expanded size of the tensor (5) must match the existing size (50) at non-singleton dimension 1.  Target sizes: [4, 5, -1].  Tensor sizes: [4, 50, 336]

In [43]:
trainer.model.style_encoder_layer

StyleEncoder(
  (layer): GST(
    (encoder): ReferenceEncoder(
      (convs): ModuleList(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (bns): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): BatchNorm2d(128,

In [ ]:
batch = trainer.format_batch(next(x for x in trainer.train_loader))
batch.keys()
input_args = [batch, trainer.criterion]
new_outs , new_loss = trainer.model.train_step(*input_args)

In [8]:
next(iter(trainer.train_loader))

BrokenPipeError: [Errno 32] Broken pipe

In [7]:
batch = next(x for x in trainer.train_loader)

BrokenPipeError: [Errno 32] Broken pipe